Exercise 7
===
Create a RNN that can learn a Reber grammer (http://www.willamette.edu/~gorr/classes/cs449/reber.html)

In [1]:
import random

seed = 42
random.seed(seed)

start_symbol = "b"
end_symbol = "e"

symbols = [start_symbol, "t", "p", "s", "x", "v", end_symbol]

def get_next_symbols(previous_symbol, current_symbol):
    if current_symbol == None and previous_symbol == None:
        return [start_symbol]
    elif current_symbol == start_symbol:
        return ["t", "p"]
    elif current_symbol == "t":
        if previous_symbol == start_symbol:
            return ["s", "x"]
        elif previous_symbol in ["p", "x", "t"]:
            return ["t", "v"]
        else:
            raise Exception("Invalid grammar.")
    elif current_symbol == "p":
        if previous_symbol == start_symbol:
            return ["t", "v"]
        elif previous_symbol == "v":
            return ["x", "s"]
        else:
            raise Exception("Invalid grammar.")
    elif current_symbol == "s":
        if previous_symbol in ["t", "s"]:
            return ["x", "s"]
        elif previous_symbol in ["x", "p"]:
            return [end_symbol]
        else:
            raise Exception("Invalid grammar.")
    elif current_symbol == "x":
        if previous_symbol in ["t", "s"]:
            return ["x", "s"]
        elif previous_symbol in ["x", "p"]:
            return ["t", "v"]
        else:
            raise Exception("Invalid grammar.")
    elif current_symbol == "v":
        if previous_symbol in ["t", "x", "p"]:
            return ["p", "v"]
        elif previous_symbol == "v":
            return [ end_symbol ]
        else:
            raise Exception("Invalid grammar.")
    elif current_symbol == end_symbol:
        return []
    else:
        raise Exception("Invalid symbols: %s and %s." % (previous_symbol, current_symbol))

def get_next_symbols_for_string(reber_str):
    previous_symbol = reber_str[-2] if len(reber_str) >= 2 else None
    current_symbol = reber_str[-1] if len(reber_str) >= 1 else None
    return get_next_symbols(previous_symbol, current_symbol)

def create_reber_string():
    reber_str = ""
    while not reber_str.endswith(end_symbol):
        reber_str += random.choice(get_next_symbols_for_string(reber_str))
    return reber_str

def is_valid_reber_string(value):
    index = 0
    while index < len(value):
        current = value[index]
        next_symbols = get_next_symbols_for_string(value[:index] if index != 0 else "")
        if current not in next_symbols:
            return False
        index += 1
    return True

In [2]:
import numpy as np
import tensorflow as tf

np.random.seed(seed)
tf.set_random_seed(seed)

def create_reber_strings(size):
    strings = []
    while len(strings) < size:
        new_string = create_reber_string()
        if new_string not in strings:
            strings.append(new_string)
    return strings

def invalidate_reber_string(value):
    while is_valid_reber_string(value):
        char_list = list(value)
        char_list[random.randint(0, len(value) - 1)] = random.choice(symbols)
        value = "".join(char_list)
    return value

def invalidate_reber_strings(strings):
    return [invalidate_reber_string(x) for x in strings]

def symbol_to_one_hot_encoding(symbol):
    one_hot_encoding = np.zeros((len(symbols)), dtype=np.int8)
    if symbol != '':
        one_hot_encoding[symbols.index(symbol)] = 1
    return one_hot_encoding

def prep_strings_for_model(strings, sequence_length):
    padded_strings = np.array([np.pad(list(x), (0, (sequence_length - len(x)) % sequence_length), 'constant') for x in strings])
    sequences = np.array([np.array(list(map(symbol_to_one_hot_encoding, x))) for x in padded_strings])
    return sequences

def generate_dataset(size, error_ratio = 0.5):
    if size % 2 != 0:
        raise Exception("size must be a multiple of 2.")
    correct_strings = create_reber_strings(int(size * error_ratio))
    incorrect_strings = invalidate_reber_strings(correct_strings)
    correct_val = True
    incorrect_val = False
    targets = np.array(([ correct_val ] * len(correct_strings)) + ([ incorrect_val ] * len(incorrect_strings)))
    indices = np.random.permutation(size)
    strings = correct_strings + incorrect_strings
    sequence_lengths = np.array([len(x) for x in strings])
    max_sequence_length = max(sequence_lengths)
    strings = prep_strings_for_model(strings, max_sequence_length)
    #print(strings[0])
    return strings[indices], sequence_lengths, targets[indices]

train_size = 10000
validation_size = int(train_size * 0.2)
test_size = int(train_size * 0.2)

all_strings, sequence_lengths, all_targets = generate_dataset(train_size + validation_size + test_size)
train_X = all_strings[:train_size]
train_seq_lengths = sequence_lengths[:train_size]
train_y = all_targets[:train_size]
validation_X = all_strings[train_size:train_size+validation_size]
validation_seq_lengths = sequence_lengths[train_size:train_size+validation_size]
validation_y = all_targets[train_size:train_size+validation_size]
test_X = all_strings[train_size+validation_size:train_size+validation_size+test_size]
test_seq_lengths = sequence_lengths[train_size+validation_size:train_size+validation_size+test_size]
test_y = all_targets[train_size+validation_size:train_size+validation_size+test_size]

#print(list(map(symbol_to_one_hot_encoding, create_reber_string())))
print("train size", len(train_X), "validation size", len(validation_X), "test size", len(test_X))
print("all_strings.shape", all_strings.shape)
#print("string shapes", list(map(lambda a: a.shape, all_strings)))
print("Example sequence:", all_strings[1], all_targets[1])
#print("sequence_lengths", sequence_lengths)

max_sequence_length = train_X[0].shape[0]
print("The max sequence length is", max_sequence_length)

train size 10000 validation size 2000 test size 2000
all_strings.shape (14000, 49, 7)
Example sequence: [[1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]] True
The max sequence length is 49


In [15]:
import math
from sklearn.base import BaseEstimator, ClassifierMixin

def he_normal_initialisation(n_inputs, n_outputs):
    stddev = np.power(2 / (n_inputs + n_outputs), 1 / np.sqrt(2))
    # truncated normal distributions limit the size of the weights, speeding up the training time.
    return tf.truncated_normal((n_inputs, n_outputs), stddev=stddev)

def he_uniform_initialisation(n_inputs, n_outputs):
    r = np.power(6 / (n_inputs + n_outputs), 1 / np.sqrt(2))
    # truncated normal distributions limit the size of the weights, speeding up the training time.
    return tf.random_uniform((n_inputs, n_outputs), -r, r)

def create_next_batch_fn(data, sequence_lengths, targets, batch_size):
    assert len(data) == len(sequence_lengths) and len(data) == len(targets)
    current_batch = 0
    def next_batch():
        nonlocal current_batch
        i = current_batch
        #print(current_batch)
        current_batch = (current_batch + batch_size) % len(data)
        return data[i:i+batch_size], sequence_lengths[i:i+batch_size], targets[i:i+batch_size]
    return next_batch

class RnnClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_steps, learning_rate=0.001, n_neurons=2):
        self.n_steps = n_steps
        self.learning_rate = learning_rate
        self.n_neurons = n_neurons
        self._build_graph()

    def _build_graph(self):
        n_inputs = 1
        self.n_output = 1
        self.batch_size = 50
        
        self.x = tf.placeholder(tf.float32, shape=(None, self.n_steps, len(symbols)), name="input")
        self.sequence_length = tf.placeholder(tf.int32, shape=(None), name="sequence_length")
        self.y = tf.placeholder(tf.bool, shape=(None), name="y")

        with tf.name_scope("rnn"):
            #cell = tf.contrib.rnn.OutputProjectionWrapper(
            #    tf.contrib.rnn.BasicRNNCell(num_units=self.n_neurons, activation=tf.nn.relu),
            #    output_size=self.n_output)
            self.rnn_activation_midpoint = 0.0
            cell = tf.contrib.rnn.GRUCell(num_units=self.n_neurons, activation=tf.nn.tanh)
            outputs, last_outputs = tf.nn.dynamic_rnn(cell, self.x, dtype=tf.float32, sequence_length=self.sequence_length)
            #print("last_outputs.shape", last_outputs.shape)
            with tf.name_scope("fc"):
                W = tf.Variable(tf.truncated_normal((self.n_neurons, self.n_output), stddev=10.1), name="weights")
                b = tf.Variable(tf.zeros([self.n_output]), name="biases")
                logits = tf.matmul(last_outputs, W) + b
                self.y_proba = tf.nn.sigmoid(logits)

        with tf.name_scope("loss"):
            y_float = tf.reshape(tf.cast(self.y, tf.float32), (-1, 1))
            self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_float, logits=logits))

        with tf.name_scope("training"):
            optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
            self.training_op = optimizer.minimize(self.loss)

        with tf.name_scope("eval"):
            self.y_pred = logits > self.rnn_activation_midpoint
            correctness = tf.equal(self.y_pred, self.y)
            self.accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32)) * 100
            
        self.init = tf.global_variables_initializer()

    def fit(self, X, sequence_lengths, y, valid_X, valid_sequence_length, valid_y, epochs = 20):
        saver = tf.train.Saver()

        interim_checkpoint_path = "./checkpoints/reber_rnn_model.ckpt"
        early_stopping_checkpoint_path = "./checkpoints/reber_rnn_model_early_stopping.ckpt"

        from datetime import datetime

        now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
        root_logdir = "tf_logs"
        log_dir = "{}/run-{}/".format(root_logdir, now)

        loss_summary = tf.summary.scalar('loss', self.loss)
        accuracy_summary = tf.summary.scalar("accuracy", self.accuracy)
        summary_op = tf.summary.merge([loss_summary, accuracy_summary])
        file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())
        
        n_batches = int(np.ceil(len(X) // self.batch_size))
        next_batch = create_next_batch_fn(X, sequence_lengths, y, self.batch_size)
            
        early_stopping_check_frequency = n_batches // 16
        early_stopping_check_limit = n_batches * 2

        sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
        self.session = sess
        sess.run(self.init)
        #saver.restore(sess, interim_checkpoint_path)

        best_validation_acc = 0.0
        best_validation_step = 0
        for epoch in range(epochs):
            print("epoch", epoch)
            for batch_index in range(n_batches):
                step = epoch * n_batches + batch_index
                X_batch, seq_length_batch, y_batch = next_batch()
                #print("training batch", X_batch.shape, seq_length_batch.shape, y_batch.shape)
                #print(seq_length_batch)
                if batch_index % 10 == 0:
                    summary_str = summary_op.eval(session=sess, feed_dict={self.x: X_batch, self.sequence_length: seq_length_batch, self.y: y_batch})
                    file_writer.add_summary(summary_str, step)
                t, l, a, proba = sess.run([self.training_op, self.loss, self.accuracy, self.y_proba], feed_dict={self.x: X_batch, self.sequence_length: seq_length_batch, self.y: y_batch})
                #print("y_proba", proba)
                if batch_index % 10 == 0: print("loss:", l, "train accuracy:", a)
                # Early stopping check
                if batch_index % early_stopping_check_frequency == 0:
                    validation_acc = self.prediction_accuracy(valid_X, valid_sequence_length, valid_y)
                    print("validation accuracy", validation_acc)
                    if validation_acc > best_validation_acc:
                        saver.save(sess, early_stopping_checkpoint_path)
                        best_validation_acc = validation_acc
                        best_validation_step = step
                    elif step >= (best_validation_step + early_stopping_check_limit):
                        print("Stopping early during epoch", epoch)
                        break
            else:
                continue
            break
            save_path = saver.save(sess, interim_checkpoint_path)
        saver.restore(sess, early_stopping_checkpoint_path)
        save_path = saver.save(sess, "./checkpoints/reber_rnn_model_final.ckpt")
            
    def predict_proba(self, X, sequence_lengths):
        dataset_size = X.shape[0]
        #print "dataset_size: ", dataset_size, " batch_size: ", batch_size
        predictions = np.ndarray(shape=(dataset_size, self.n_output), dtype=np.float32)
        steps = int(math.ceil(dataset_size / self.batch_size))
        #print "steps: ", steps
        for step in range(steps):
            offset = (step * self.batch_size)
            #print "offset ", offset
            data_end_index = min(offset + self.batch_size, dataset_size)
            batch_data = X[offset:data_end_index, :]
            feed_dict = {
                self.x: batch_data,
                self.sequence_length: sequence_lengths[offset:data_end_index]
            }
            predictions[offset:data_end_index, :] = self.y_proba.eval(session=self.session, feed_dict=feed_dict)
        #print("predict_proba", predictions)
        return predictions

    def predict(self, X, sequence_lengths):
        return np.argmax(self.predict_proba(X, sequence_lengths), axis=1)
    
    def _prediction_accuracy(self, predictions, y):
        probability_midpoint = 0.5
        return (np.sum(((predictions.reshape((-1)) > probability_midpoint) == y).astype(float))
              / predictions.shape[0]) * 100
    
    def prediction_accuracy(self, X, sequence_lengths, y):
        predictions = self.predict_proba(X, sequence_lengths)
        return self._prediction_accuracy(predictions, y)

In [ ]:
tf.reset_default_graph()
rnn_classifier = RnnClassifier(max_sequence_length, learning_rate=0.1, n_neurons=10)

rnn_classifier.fit(train_X, train_seq_lengths, train_y, validation_X, validation_seq_lengths, validation_y)

epoch 0
loss: 3.99558 train accuracy: 58.0
validation accuracy 52.1
loss: 0.730568 train accuracy: 50.8
validation accuracy 48.7
loss: 0.71854 train accuracy: 51.6
validation accuracy 49.15
loss: 0.712743 train accuracy: 51.6
validation accuracy 53.95
loss: 0.684097 train accuracy: 51.92
validation accuracy 53.95
loss: 0.69673 train accuracy: 48.32


In [ ]:
subset_train_acc = rnn_classifier.prediction_accuracy(train_X[:20], train_seq_lengths[:20], train_y[:20])
print(">>>> First 20 training instances accuracy", subset_train_acc)


test_acc = rnn_classifier.prediction_accuracy(test_X, test_seq_lengths, test_y)
print(">>>> Test dataset accuracy:", test_acc)